## Practical session 3: First-order and second-order algorithms on a practical problem

In this session, we will investigate how to evaluate and display performance of optimization algorithms over a practical problem of machine learning and statistics: binary classification using logistic regression. The considered algorithms will be gradient descent algorithms, possibly with line-search, but we will also introduce second-order algorithms.

### From statistics and machine learning to optimization

#### Supervised learning

We have some *data*  $\mathcal{D}$ consisting of $m$ *examples* $\{d_i\}$; each example consisting of a *feature* vector $a_i\in\mathbb{R}^d$ and an *observation* $b_i\in \mathcal{O}$: $\mathcal{D} = \{[a_i,b_i]\}_{i=1}^m$. In this lab, we will consider the <a href="https://archive.ics.uci.edu/ml/machine-learning-databases/ionosphere/ionosphere.names">ionosphere</a> dataset.
 
The goal of *supervised learning* is to construct a predictor for the observations when given feature vectors.

A popular approach is based on *linear models* which are based on finding a *parameter* $x$ such that the real number $\langle a_i , x \rangle$ is used to predict the value of the observation through a *predictor function* $g:\mathbb{R}\to \mathcal{O}$: $g(\langle a_i , x \rangle)$ is the predicted value from $a_i$.

In order to find such a parameter, we use the available data and a *loss* $\ell$ that penalizes the error made between the predicted $g(\langle a_i , x \rangle)$ and observed $b_i$ values. For each example $i$, the corresponding error function for a parameter $x$ is $f_i(x) =   \ell( g(\langle a_i , x \rangle) ; b_i )$. Using the whole data, the parameter that minimizes the total error is the solution of the minimization problem

$$ \min_{x\in\mathbb{R}^d}  \frac{1}{m} \sum_{i=1}^m f_i(x) = \frac{1}{m} \sum_{i=1}^m  \ell( g(\langle a_i , x \rangle) ; b_i ). $$

#### Binary classification with logistic regression

In our setup, the observations are binary: $\mathcal{O} = \{-1 , +1 \}$, and the *Logistic loss* is used to form the following optimization problem
\begin{align*}
\min_{x\in\mathbb{R}^d } f(x) := \frac{1}{m}  \sum_{i=1}^m  \log( 1+\exp(-b_i \langle a_i,x \rangle) ) + \frac{\lambda_2}{2} \|x\|_2^2.
\end{align*}
where the last term is added as a regularization (of type $\ell_2$, aka Tikhonov) to prevent overfitting.

This optimization-based formulation is grounded in statistics: when $\lambda_2 = 0$, $x_* = \arg \min f$ can be seen a maximum likelihood estimator (with the probability of being in class , while when $\lambda_2 > 0$, $x_*$ is a maximum a posteriori estimator with a Gaussian prior. Then, for a new point $d$ with features vector $a$, 
$$ p_1(a) = \mathbb{P}[d\in \text{ class }  +1] = \frac{1}{1+\exp(-\langle a,x^\star \rangle)} $$

Thus, from $a$, if $p_1(a)$ is close to $1$, one can decide that $d$ belongs to class $1$; and the opposite decision if $p(a)$ is close to $0$. Between the two, the appreciation is left to the data scientist depending on the application.

### Solving logistic regression with first-order methods

Several gradient descent algorithms are implemented in the file `algorithms.py`, namely, gradient descent with fixed step size and with Wolfe line-search. The file `logistic_regression_ionosphere.py` contains all the oracles of $f$ (function, gradient, and Hessian) as well as the Lipschitz constant of $\nabla f$, allowing to choose step sizes that will lead to convergence of gradient descent algorithms with fixed step size.

In [ ]:
from algorithms import *  
import logistic_regression_ionosphere as pb

prec = 1e-5              
iterMax = 5000                     
x_init = np.zeros(pb.n)              

> Run the following block for different values of parameter `lam2` of the problem. In particular, try $\lambda_2 \in \{0, 0.01, 0.1, 1\}. What do you notice in terms of speed of convergence, what is the reason?

In [ ]:
pb.lam2 = 0.0
L_constant = pb.L + pb.lam2
tau = 1.0/L_constant

##### gradient algorithm
x,x_tab = GD(pb.f , pb.f_grad , x_init , tau , iterMax , prec )
plot_obj_normGrad(x_tab, pb.f, pb.f_grad, r"Convergence of GD ($\tau = 1/L$) on $f$ with $\lambda_2 = 0$")

In [ ]:
### TO BE COMPLETED

In [ ]:
### TO BE COMPLETED

In [ ]:
### TO BE COMPLETED

### Second-order methods

So far, we have only considered algorithms of the form $x_{k+1} = x_k - \tau_k d_k$, with $d_k = \nabla f(x_k)$. However, it is possible to choose the descent direction $d_k$ differently, in particular, second-order information about $f$ can be leveraged to create more efficient algorithms. We are now going to introduce two algorithms using recursions of the form $x_{k+1} = x_k - \tau_k W_k \nabla f(x_k)$ where $W_k$ is a matrix aiming at capturing the curvature of $f$ and thus improving convergence.

#### Newton's method

Newton's method is usually used to find root of functions. In the context of optimization, the Newton's method corresponds to applying the root-finding Newton's method to find the roots of $\nabla f$. This leads to the following recursion at iteration $k \in \mathbb{N}$:
$$
x_{k+1} = x_k - (\nabla^2 f(x_k))^{-1} \nabla f(x_k).
$$

> Run Newton's method using the `newton` function provided in `algorithms.py', for $\lambda_2 \in \{0.01, 0.1, 1\}$. Compare the convergence to the one obtained with the gradient descent algorithm. What can you say?

In [ ]:
pb.lam2 = 0.01

##### gradient algorithm
x,x_tab = newton(pb.f , pb.f_grad_hessian , x_init , prec , iterMax)
plot_obj_normGrad(x_tab, pb.f, pb.f_grad, r"Convergence of Newton's method on $f$")

In [ ]:
### TO BE COMPLETED

In [ ]:
### TO BE COMPLETED

> What happens when running the Newton's method with $\lambda_2 = 0$?

> Try running the Newton's algorithm with $\lambda_2 = 0.1$ starting from the vector containing only the value $0.5$. What happens?

In [ ]:
### TO BE COMPLETED

#### The BFGS algorithm

Newton's algorithm benefits from very attractive properties which only hold in restricted settings. Namely, if the algortihm is initialized close enough to a solution and under sufficient regularity of $f$, then the convergence is super-linear.

However, Newwton's method suffers from several drawbacks:
- It requires the computation and inversion of a $n \times n$ matrix at every iteration,
- The convergence is only local,
- $d_k = (\nabla^2 f(x_k))^{-1} \nabla f(x_k)$ is not necessarily a descent direction.


In order to adress the computational cost of Newton's method, many methods of the so-called quasi-Newton type have been proposed, where a matrix $W_k$ is constructed as an approximation of $(\nabla^2 f(x_k))^{-1}$ using a cheaper process.

The most prominent quasi-Newton method is the BFGS algorithm (Broyden-Fletcher-Goldfarb-Shanno, 1970), whose iterations can be described as
$ x_{k+1}=x_k - \tau_k (M_k)^{-1} \nabla f(x_k)$, where $\tau_k$ is given by Wolfe's line-search and positive definite matrix $M_k$ is computed as
$$ M_{k+1}=M_k - \frac{s_k y_k^T M_k+M_k y_k s_k^T}{y_k^T s_k} +\left[1+\frac{y_k^T M_k y_k}{y_k^T s_k}\right]\frac{s_k s_k^T}{y_k^T s_k} $$
with $s_k=x_{k+1}-x_{k}$ and $y_k=\nabla f(x_{k+1}) - \nabla f(x_{k})$. The matrices $M_k$ are approximations of the Hessian in the sense that $y_k = M_k s_k$, $M_k = M_k^\top$, and successive $M_{k+1}$ and $M_k$ are close in some sense.

This formula bypasses accessing the Hessian, but still requires matrix inversion. Fortunately, a similar recurrence can be derived for the sequences of inverses $W_k = M_k^{-1}$, with
$$
W_{k+1} = \left( I - \frac{s_k y_k^\top}{y_k^\top s_k} \right) W_k \left( I - \frac{y_k s_k^\top}{y_k^\top s_k} \right) + \frac{s_k s_k^\top}{y_k^\top s_k}.
$$
As you can see, BFGS allows to bypass Hessian access and matrix inversion completely compared to Newton's method. Indeed, one can just compute $d_k = W_k \nabla f(x_k)$ and implement $x_{k+1} = x_k - \tau_k d_k$. 



> Implement the BFGS algorithm in the file `algorithms.py`. The functions `np.inner`, `np.outer`, and `np.eye` may be useful.

> Implement the BFGS algorithms in `algoGradient.py`.

> Compare the performance of the previously investigated algorithms using the following blocks of code.

In [ ]:
from algorithms import *  


prec = 1e-5          
iterMax = 500                  
x_init = np.zeros(pb.n)         

pb.lam2 = 0.01
L_constant = pb.L + pb.lam2
tau = 1.0/L_constant

##### gradient descent algorithm with fixed step size
x,x_tab = GD(pb.f , pb.f_grad , x_init , tau , iterMax , prec)

##### gradient descent with Wolfe line-search algorithm
xW,xW_tab = GD_wolfe(pb.f , pb.f_grad , x_init , prec , iterMax )

##### Newton's method
xN,xN_tab = newton(pb.f , pb.f_grad_hessian , x_init , prec , iterMax )

##### BFGS algorithm
xB,xB_tab = bfgs(pb.f , pb.f_grad , x_init , prec , iterMax )


In [ ]:
import matplotlib.pyplot as plt


F = []
G = []
for i in range(x_tab.shape[0]):
    F.append( pb.f(x_tab[i])) 
    G.append( np.linalg.norm(pb.f_grad(x_tab[i] )) )
   
FW = []
GW = []
for i in range(xW_tab.shape[0]):
    FW.append( pb.f(xW_tab[i])) 
    GW.append( np.linalg.norm(pb.f_grad(xW_tab[i] )) )

FN = []
GN = []
for i in range(xN_tab.shape[0]):
    FN.append( pb.f(xN_tab[i])) 
    GN.append( np.linalg.norm(pb.f_grad(xN_tab[i] )) )
 
    
FB = []
GB = []
for i in range(xB_tab.shape[0]):
    FB.append( pb.f(xB_tab[i])) 
    GB.append( np.linalg.norm(pb.f_grad(xB_tab[i] )) )

plt.figure()
plt.plot( F, color="black", linewidth=1.0, linestyle="-",label='gradient')
plt.plot( FW, color="magenta", linewidth=1.0, linestyle="-",label='Wolfe')
plt.plot( FN, color="red", linewidth=1.0, linestyle="-",label='Newton')
plt.plot( FB, color="green", linewidth=1.0, linestyle="-",label='BFGS')
plt.grid(True)
plt.legend()
plt.show()


plt.figure()
plt.plot( G, color="black", linewidth=1.0, linestyle="-",label='gradient')
plt.plot( GW, color="magenta", linewidth=1.0, linestyle="-",label='Wolfe')
plt.plot( GN, color="red", linewidth=1.0, linestyle="-",label='Newton')
plt.plot( GB, color="green", linewidth=1.0, linestyle="-",label='BFGS')
plt.yscale('log')
plt.xscale('log')
plt.grid(True)
plt.legend()
plt.show()


### To go further: Performance on learning problems


Our problem of interest is binary classification using logistic regression.</br>
Although this is a machine learning task, the predictor construction amounts to minimizing a smooth convex optimization function $f$ called the *loss*, the final minimizer is called a *predictor* and its scalar product with the data vector gives a probability of belonging to class $1$.

The previous test was based on the functional decrease whereas our task is binary classification. Let us look at the final accuracies obtained.

> The file `logistic_regression.py` contains a `prediction` function that takes a *predictor* and resturn the accuracy of the predictor. Take a look at how the function is defined.

> Observe the accuracy of all final points obtained before. What do you notice? 

In [ ]:
pred,perf = pb.prediction(x,PRINT=False)
print("Gradient algorithm: \t{:.2f}%".format(perf*100))


predW,perfW = pb.prediction(xW,PRINT=False)
print("Wolfe: \t\t\t{:.2f}%".format(perfW*100))

predN,perfN = pb.prediction(xN,PRINT=False)
print("Fast Gradient: \t\t{:.2f}%".format(perfN*100))


predB,perfB = pb.prediction(xB,PRINT=False)
print("BFGS: \t\t\t{:.2f}%".format(perfB*100))

### To go further: Globalizing Newton's method convergence

The convergence of Newton's method is only local and requires a careful initialization. A possible way to "globalize" the convergence is to add a step size at every iteration that is selected by line search. More explicitly, if the standard Newton's update is of the form $x_{k+1} = x_k - d_k$ with $d_k = (\nabla^2 f(x_k))^{-1} \nabla f(x_k)$, then we will perform $x_{k+1} = x_k - \tau_k d_k$ where $\tau_k$ is selected by Wolfe's line-search method.

> Implement this method and see if it allows for Newton's method to converge even on initial points where the standard Newton's method failed.

In [ ]:
### TO BE COMPLETED